<h1>Chap07 - Handling Dates and Times</h1>

Imports

In [17]:
import numpy as np
import pandas as pd
from pytz import all_timezones

## 7.1 Converting Strings to Dates 

In [3]:
date_strings = np.array(['03-04-2005 11:35 PM',
                         '23-05-2010 12:01 AM',
                         '04-09-2009 09:09 PM'])

In [4]:
[pd.to_datetime(date, format='%d-%m-%Y %I:%M %p') for date in date_strings]

[Timestamp('2005-04-03 23:35:00'),
 Timestamp('2010-05-23 00:01:00'),
 Timestamp('2009-09-04 21:09:00')]

In [5]:
[pd.to_datetime(date, format='%d-%m-%Y %I:%M %p', errors = 'coerce') for date in date_strings]

[Timestamp('2005-04-03 23:35:00'),
 Timestamp('2010-05-23 00:01:00'),
 Timestamp('2009-09-04 21:09:00')]

## 7.2 Handling Timezones 

In [6]:
pd.Timestamp('2017-05-01 06:00:00', tz='Europe/London')

Timestamp('2017-05-01 06:00:00+0100', tz='Europe/London')

In [9]:
date = pd.Timestamp('2017-05-01 06:00:00')
date_in_london = date.tz_localize('Europe/London')
date_in_london

Timestamp('2017-05-01 06:00:00+0100', tz='Europe/London')

In [11]:
date_in_london.tz_convert('Africa/Abidjan')

Timestamp('2017-05-01 05:00:00+0000', tz='Africa/Abidjan')

In [15]:
#Create dates
dates = pd.Series(pd.date_range('2/2/2002', periods=3, freq='M'))

#Set time zone
dates.dt.tz_localize('Africa/Abidjan')

0   2002-02-28 00:00:00+00:00
1   2002-03-31 00:00:00+00:00
2   2002-04-30 00:00:00+00:00
dtype: datetime64[ns, Africa/Abidjan]

In [18]:
all_timezones[0:2]

['Africa/Abidjan', 'Africa/Accra']

## 7.3 Selecting Dates and Times 

In [19]:
dataframe = pd.DataFrame()
dataframe['date'] = pd.date_range('1/1/2001', periods=100000, freq='H')
dataframe[(dataframe['date'] > '2002-1-1 01:00:00') & (dataframe['date'] <= '2002-1-1 04:00:00')]

,date
8762,2002-01-01 02:00:00
8763,2002-01-01 03:00:00
8764,2002-01-01 04:00:00


In [21]:
dataframe = dataframe.set_index(dataframe['date'])

In [22]:
dataframe['2002-1-1 01:00:00':'2002-1-1 04:00:00']

,date
date,
2002-01-01 01:00:00,2002-01-01 01:00:00
2002-01-01 02:00:00,2002-01-01 02:00:00
2002-01-01 03:00:00,2002-01-01 03:00:00
2002-01-01 04:00:00,2002-01-01 04:00:00


## 7.4 Breaking Up Date Data into Multiple Features 

In [25]:
dataframe = pd.DataFrame()
dataframe['date'] = pd.date_range('1/1/2001', periods=150, freq='W')

dataframe['year'] = dataframe['date'].dt.year
dataframe['month'] = dataframe['date'].dt.month
dataframe['day'] = dataframe['date'].dt.day
dataframe['hour'] = dataframe['date'].dt.hour
dataframe['minute'] = dataframe.date.dt.minute

dataframe.head(3)

,date,year,month,day,hour,minute
0,2001-01-07,2001,1,7,0,0
1,2001-01-14,2001,1,14,0,0
2,2001-01-21,2001,1,21,0,0


## 7.5 Calculating the Differences Between Dates

In [26]:
dataframe = pd.DataFrame()

In [27]:
dataframe['Arrived'] = [pd.Timestamp('01-01-2017'), pd.Timestamp('01-04-2017')]
dataframe['Left'] = [pd.Timestamp('01-01-2017'), pd.Timestamp('01-06-2017')]

In [30]:
dataframe['Left'] - dataframe['Arrived']

0   0 days
1   2 days
dtype: timedelta64[ns]

In [31]:
# Calculate duration between features
pd.Series(delta.days for delta in (dataframe['Left'] - dataframe['Arrived']))

0    0
1    2
dtype: int64

## 7.6 Encoding Days of the Week 

In [36]:
dates = pd.Series(pd.date_range('2/2/2002', periods=3, freq='M'))
dates.dt.day_name()

0    Thursday
1      Sunday
2     Tuesday
dtype: object

In [38]:
dates.dt.dayofweek

0    3
1    6
2    1
dtype: int64

## 7.7 Creating a Lagged Feature 

In [40]:
dataframe = pd.DataFrame()
dataframe['dates'] = pd.date_range('1/1/2001', periods = 5, freq='D')
dataframe['stock_price'] = [1.1, 2.2, 3.3, 4.4, 5.5]
dataframe['previous_days_stock_price'] = dataframe['stock_price'].shift(1)

In [41]:
dataframe

,dates,stock_price,previous_days_stock_price
0,2001-01-01,1.1,NaN
1,2001-01-02,2.2,1.1
2,2001-01-03,3.3,2.2
3,2001-01-04,4.4,3.3
4,2001-01-05,5.5,4.4


## 7.8 Using Rolling Time Windows 

In [42]:
time_index = pd.date_range('01/01/2010', periods=5, freq='M')
dataframe = pd.DataFrame(index=time_index)
dataframe['Stock_Price'] = [1, 2, 3, 4, 5]
dataframe.rolling(window=2).mean()

,Stock_Price
2010-01-31,NaN
2010-02-28,1.5
2010-03-31,2.5
2010-04-30,3.5
2010-05-31,4.5


## 7.9 Handling Missing Data in Time Series 

In [49]:
dataframe = pd.DataFrame(index = time_index)
dataframe['Sales'] = [1.0, 2.0, np.nan, np.nan, 5.0]
dataframe.interpolate()

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,3.0
2010-04-30,4.0
2010-05-31,5.0


In [50]:
dataframe = pd.DataFrame(index = time_index)
dataframe['Sales'] = [1.0, 2.0, np.nan, np.nan, 5.0]
dataframe.ffill()

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,2.0
2010-04-30,2.0
2010-05-31,5.0


In [51]:
dataframe = pd.DataFrame(index = time_index)
dataframe['Sales'] = [1.0, 2.0, np.nan, np.nan, 5.0]
dataframe.bfill()

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,5.0
2010-04-30,5.0
2010-05-31,5.0


In [52]:
dataframe = pd.DataFrame(index = time_index)
dataframe['Sales'] = [1.0, 2.0, np.nan, np.nan, 5.0]
dataframe.interpolate(method='quadratic')

,Sales
2010-01-31,1.000000
2010-02-28,2.000000
2010-03-31,3.059808
2010-04-30,4.038069
2010-05-31,5.000000


In [53]:
dataframe = pd.DataFrame(index = time_index)
dataframe['Sales'] = [1.0, 2.0, np.nan, np.nan, 5.0]
dataframe.interpolate(limit=1, limit_direciton='forward')

,Sales
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,3.0
2010-04-30,NaN
2010-05-31,5.0
